## Import necessary python modules

In [1]:
import os
import napari
import tifffile
import numpy as np
import matplotlib.pyplot as plt

import bigfish
import bigfish.plot as plot
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack

from copy import deepcopy
from dask.array.image import imread as imr
from bigfish.detection.utils import get_object_radius_pixel
from buildReferenceSpot import buildReferenceSpotFromImages
from runBigfishDetection import getSpotAndClusters, saveSpotsNPZ, reorderZstack, getSpotAndClusters_multi

In [2]:
from order_spot import * 

In [3]:
import dask.array as da
def verify_array(labels):
    '''
    Convert labels to numpy array
    '''
    if not isinstance(labels, np.ndarray):

        if isinstance(labels,list):
            labels = np.array(labels)

        if isinstance(labels, da.core.Array):
            labels = labels.compute()

    return labels


def generate_background(movie_frame, background_signal,min_val, max_val):
    
    '''
    This function generate background for cells image according to a list of background signals 
    '''
    res =[]
    movie_frame=verify_array(movie_frame)
    
    aleatoire = np.random.uniform(min_val,max_val,movie_frame.shape)
    res=np.where(movie_frame==background_signal, aleatoire,movie_frame)

    return res 

## Specify Input and Output folders, identifiers

In [4]:
homeFolder = '/media/raphael/data/works/20231220_C3_14_2h_ofON/bleach_correction/expo/'
nameKey = 'C3.14_ON_01_w1SPI-488_s'

imsQ = '4'
cellNumber = '54'
nucleiStackForm = nameKey+imsQ+"_cell_"

pathToTimeSequenceFullView = homeFolder+nameKey+imsQ+'/*.tif'
cell_Crop_Folder = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/'
pathToTimeSequenceCell = homeFolder+nameKey+imsQ+'/cell_'+str(cellNumber)+'/*.tif'


sequenceCell = imr(pathToTimeSequenceCell)
sequenceCell = generate_background(sequenceCell,0,100,160)
mipSequenceCell = np.max(sequenceCell, axis=1)
MaxTimePoint = sequenceCell.shape[0]

## Specify voxel and object size

In [5]:
voxelRadius = (700, 110, 110) # in nanometers
objectRadius = (700, 105, 105) # in nanometers

In [6]:
alpha = 0.9

beta = 2
gamma = 15

## Choose detection threshold

In [7]:
selectedThreshold = int(input("Enter your selected threshold: "))

Enter your selected threshold: 45


In [8]:
minFrame = int(input("Enter your min Frame: "))
#minFrame = minFrame 

Enter your min Frame: 1


In [9]:
maxFrame = int(input("Enter your max Frame: "))
maxFrame = maxFrame + 1 

Enter your max Frame: 120


### Compute list of threshold 

## Get a list of spots detected using the threshold specified in previous step

Voire graphe du notebook 4 bleaching correction

Toutes les frames ne suivent pas la meme allure avec des images bcp moins intense que l'autre d'une frame a l'autre besoin de ponderer le threshold en fonction d'une moyenne generale et d'une liste de moyenne en fonction des frames 

if mean_frame < moy_gen :
    abaisser le threshold
elid mean_frame > moy_gen :
    augmenter le threshold
    
else : 
    conserver threshold 
    
Ou possibilite d'appliquer ce principe a une difference de pourcentage d'une frame a l'autre 

In [10]:
images=[]    
spots_list=[]

spot_radius_px = detection.get_object_radius_pixel(
    voxel_size_nm=voxelRadius, 
    object_radius_nm=objectRadius, 
    ndim=3)

for t in range(minFrame,maxFrame,1):

    path = os.path.join(cell_Crop_Folder, nucleiStackForm+str(cellNumber)+'_t'+str(f"{t:03}")+".tif")
    rna = stack.read_image(path)
    images.append(rna)

images =  generate_background(images,0,100,160)
images.tolist()

n=len(images)
print("Total number of images : "+str(n))

selectedThreshold_loop = selectedThreshold

for rna in images:
    try : 
        # LoG filter
        rna_log = stack.log_filter(rna, sigma=spot_radius_px)
    
        # local maximum detection
        mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)
    
        # thresholding
        threshold = detection.automated_threshold_setting(rna_log, mask)
        
        selectedThreshold_loop = (selectedThreshold_loop) #*((100+var_list[i])/100) 
        #print(selectedThreshold_loop)
        spots_, _ = detection.spots_thresholding(rna_log, mask, float(selectedThreshold_loop))
        spots_list.append(spots_)
    except IndexError as e: 
        spots_, _ = detection.spots_thresholding(rna_log, mask, float(selectedThreshold))
        spots_list.append(spots_)
        print('Done')
        break 

Total number of images : 120


In [ ]:
images.tolist()

## Build reference spot

In [ ]:
reference_spot = buildReferenceSpotFromImages(images, spots_list, alpha=alpha, gamma=5, voxelSize=voxelRadius, objectSize=objectRadius)
%matplotlib inline
plt.figure(num=3,figsize=(3,3))
plt.imshow(np.max(reference_spot, axis=0))
refSpot = deepcopy(reference_spot)

## Perform spot and cluster detection for all frames

In [ ]:
spotsFrame, clustersFrames, ThresholdFrames = getSpotAndClusters(images, 
                                                                 reference_spot, 
                                                                 cellnumber=cellNumber, 
                                                                 startTime=minFrame-1,
                                                                 stopTime=maxFrame-1,
                                                                 thresholdManual=selectedThreshold, 
                                                                 beta=beta, #2
                                                                 gamma=gamma, #15 
                                                                 numberOfSpots=2,
                                                                 radiusCluster=400, 
                                                                 voxelSize=voxelRadius, 
                                                                 objectSize=objectRadius,
                                                                 extensionMov='.tif')

In [ ]:
def sort_clustersFrames(clustersFrames,mipSequenceCell,threshold,perc=1):
    '''This function sort a list of numpy array according to a threshold corresponding to the intensity of the mipSequenceCell's pixels 
    '''

    clustersFrames_cp = deepcopy(clustersFrames)
    res = []
    i = 0 
    with alive_bar(len(clustersFrames_cp), title = "sort cluster frame",force_tty=True) as bar : 
        for frames in clustersFrames_cp: 
            tmp = []
            tmp_res = []
            for cand in frames :
                y = cand[1]
                x = cand[2]
                
                test = np.array(mipSequenceCell[i][y:y+1, x:x+1])
                
                val = test[0][0]
                #if val < threshold: 
                    #print(val)
                tmp.append(val) 
            #print(i, np.array(tmp))
            
            
            try : 
                if max(tmp)<threshold:
                    res.append([np.zeros(5)])
                    
                else : 
                    # print('i = ', i,'len frames = ', int(len(frames)*perc))
                    for j in range(0,int(len(frames)*perc)):
                        index_max = tmp.index(max(tmp)) #index of most intense point of images 
                        fmax = deepcopy(frames[index_max]) # get corresponding clusters coordinates 
                        cp = compare_array(fmax,tmp_res)

                        #print(i,j, fmax, frames, index_max)
                        if np.any(cp)!= True :
                            #print("!= True")
                            tmp_res.append(fmax)
                        tmp.pop(index_max) # pop max point 
                        
                        
                        frames = frames.tolist()
                        frames.pop(index_max) # pop corresponding clusters 
                        frames = np.array(frames)
                        #print('after \n ', frames)
                    res.append(tmp_res)
            except ValueError as e : 
                #print(i, tmp, test)
                print(e)
                res.append([np.zeros(5)])
            time.sleep(0.1)
            bar()
            i = i+1 
        return res 

In [ ]:
a = sort_clustersFrames(clustersFrames,mipSequenceCell,400,perc=1)

In [ ]:
clustersFrames[11]

In [ ]:
a[11]

## Correct clusters position if needed 

In [ ]:
#a[2] = move_sorted_clustersFrames(a[2], np.array(a[2][1]), 0) #position 1 


## Merge clusters 

In [ ]:
def merge_clusters_framev2(res,mipSequenceCell,diff,perc=0.5):
    '''
    This function compute coordinate of clusters Frame with the compatible neighboor to get a better quantification 
    '''
    # prendre en compte 
    res2 = deepcopy(res)
    p = 0
    i=0
    new_clusters_frames = []
    diff2 = diff * (1+(1/3))
    
    for frames in res2 :
        tc = frames[0]
        tmp = [tc]
        
        threshold = np.array(mipSequenceCell[i][tc[1]][tc[2]])*perc
        #print(i, threshold)
        for cluster in frames[1:]:
            # define tests in 3 dimensions
            test_z = tc[0] - cluster[0]
            test_y = tc[1] - cluster[1]
            test_x = tc[2] - cluster[2]

            # define test on intensity 
            test_array_a = np.array(mipSequenceCell[i][cluster[1]][cluster[2]])
            test_array_b =  np.array(mipSequenceCell[i][tc[1]][tc[2]])
            test_array =  int(test_array_a) - int(test_array_b) 

            if -diff <=test_y <= diff and -diff <=test_x <= diff and -diff <= test_z <= diff :
                if -threshold <= test_array <= +threshold : 
                    tmp.append(cluster)
                    
        
        for t in tmp[1:] :
            #average the coordinate of the clusters 
            tc[0] = (tc[0]+t[0])/2 
            tc[1] = (tc[1]+t[1])/2
            tc[2] = (tc[2]+t[2])/2

            # sum of the molecules 
            tc[3] = tc[3] + t[3]
        i=i+1
        new_clusters_frames.append([tc])
    return new_clusters_frames

In [ ]:
b = merge_clusters_framev2(a,mipSequenceCell,diff=5)

In [ ]:
c = create_index2(b)    

In [ ]:
c

## Save detection results to be used later

In [ ]:
saveSpotsNPZ(np.array(spotsFrame, dtype=object), 
             np.array(clustersFrames, dtype=object), 
             np.array(ThresholdFrames, dtype=object), 
             cellNumber, 
             cell_Crop_Folder, 
             reference_spot,
             threshold = selectedThreshold
             ) 


## Visualise detection results

In [ ]:
def getDetectedPointsForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-1])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:]
    return pts_coords

def getDetectedClustersForFrame(pts_coordinates, frameNumer):
    sd = np.shape(pts_coordinates[frameNumer][:])
    pts_coords = np.empty([sd[0],sd[1]-3])
    for ii in range(np.shape(pts_coordinates[frameNumer][:])[0]):
        pts_coords[ii,:] = pts_coordinates[frameNumer][ii][1:3]
    return pts_coords

def set_pts_features(pts_layer, cls_layer, pts_coordinates, cluster_coordinate, step): #TxLayer
    # step is a 4D coordinate with the current slider position for each dim
    frameNumber = step[0]  # grab the leading ("time") coordinate
    pts_layer.data = getDetectedPointsForFrame(pts_coordinates,frameNumber)
    cls_layer.data = getDetectedClustersForFrame(cluster_coordinate,frameNumber)


pts_coordinates = spotsFrame
cluster_coordinate = c

viewer = napari.Viewer()
image_layer = viewer.add_image(
        mipSequenceCell, colormap='green' #maxImageCell
        )
if image_layer.data.ndim == 4:
    bigfishSpots = spotsFrame
elif image_layer.data.ndim == 3:
    bigfishSpots = getDetectedPointsForFrame(pts_coordinates,int(np.shape(mipSequenceCell)[0]/2))
    
bigfish_Spots = viewer.add_points(
        getDetectedPointsForFrame(pts_coordinates,int(np.shape(mipSequenceCell)[0]/2)-1),
        face_color='#00000000',
        size=4,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='white',
        name = 'bigFish Detected Spots'
        )

bigfish_clusters = viewer.add_points(
        getDetectedClustersForFrame(cluster_coordinate,int(np.shape(mipSequenceCell)[0]/2)-1),
        face_color='#00000000',
        size=8,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='red',
        symbol='diamond',
        name = 'bigFish Clusters'
        )


viewer.dims.events.current_step.connect(
        lambda event: set_pts_features(bigfish_Spots, bigfish_clusters, pts_coordinates, cluster_coordinate, event.value)
        )



# Save clusters frame in csv 

In [ ]:
b = np.concatenate(c)
savepath = homeFolder+nameKey+imsQ + '/' + nameKey + imsQ + '_TS' +'_cell_' + cellNumber + '_fusionv2_bleach_correction'+  str(alpha) +'_' + str(beta) + '_' + str(gamma) + '_thres_' + str(selectedThreshold) + '.csv'

In [ ]:
entete = ['z','y','x','n_molecule','index']

import csv 

with open(savepath, 'w', newline='') as csv_file :
    writer = csv.writer(csv_file)
    writer.writerow(entete)
    writer.writerows(b)

In [ ]:
import session_info
session_info.show()